# AgriMarket

## 농수산물


- Data Source : [농식품 빅데이터 거래소](https://kadx.co.kr/product/detail/94534d90-595d-11eb-acdd-9560621d9652)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 일반농산물 도매 69품목 116품종, 소매 90품목 143품종 및 친환경농산물 38품목 44종 대상 가격 자료


## 소비자 물가지수 


- Data Source : [국가통계포털](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1J20112&conn_path=I2)

- Collection Method : Download

- Data Format : CSV

### 데이터 설명

- 농산물 중 필수재라고 판단되는 곡물,채소 포함

- 사치재인 과실, 기타 농산물 제외

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

root_path = "../"
data_path = f"{root_path}data"

col_detail={
'prce_reg_ymd' : '가격등록일자',
'mrkt_esnt_no' : '시장고유번호',
'mrkt_code' : '시장코드',
'mrkt_nm' : '시장명',
'ct_code' : '시도코드',
'ctnp_nm' : '시도명',
'sggu_code' : '시군구코드',
'sggu_nm' : '시군구명',
'pdlt_code' : '품목코드',
'pdlt_nm' : '품목명',
'spcs_code' : '품종코드',
'spcs_nm' : '품종명',
'wsrt_exmn_se_code' : '도소매조사구분코드',
'exmn_se_nm' : '조사구분명',
'bulk_grad_code' : '산물등급코드',
'bulk_grad_nm' : '산물등급명',
'pdlt_prce' : '품목가격',
'mtc_smt_unit_mg' : '산지출하단위크기',
'mtc_smt_unit_nm' : '산지출하단위명',
'whsl_smt_unit_mg' : '도매출하단위크기',
'whsl_smt_unit_nm' : '도매출하단위명',
'rtsl_smt_unit_mg' : '소매출하단위크기',
'rtsl_smt_unit_nm' : '소매출하단위명',
'evfd_fmpd_smt_unit_mg' : '친환경농산물출하단위크기',
'evfd_fmpd_smt_unit_nm' : '친환경농산물출하단위명',
'dcnt_prce_yn' : '할인가격여부',
'etl_ldg_dt' : 'ETL적재일시'
}

# 파일 호출, 변수명 및 불러오기 경로는 자신에게 맞게 변경해주시면 됩니다.

In [36]:
import os
files_path = f'{data_path}/TB_TAT_KAMIS_PRC_MK_TB/'
files = os.listdir(files_path)
files.remove('.DS_Store')
print(files)

['TB_TAT_KAMIS_PRC_MK_TB-2019.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2018.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2008.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2020.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2009.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2010.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2011.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2005.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2013.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2007.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2006.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2012.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2016.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2017.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2015.csv', 'TB_TAT_KAMIS_PRC_MK_TB-2014.csv']


In [37]:
df = pd.DataFrame()
for file in files:
    df_sub = pd.read_csv(f'{files_path}/{file}', encoding='cp949', low_memory=False)
    df_sub = df_sub[['PRCE_REG_YMD', 'CTNP_NM', 'PDLT_NM', 'EXMN_SE_NM', 'BULK_GRAD_NM', 'PDLT_PRCE']]
    df_sub = df_sub[~df_sub['PDLT_NM'].isna()]
    df_sub.columns=['YMD', 'Region', 'Product', 'WR', 'Grade', 'Price']
    df = pd.concat([df, df_sub])

# 1. 년·월·일 에서 연·월 로 변경
# 2. Y(년), M(월) 컬럼 생성

In [82]:
df['YMD'] = df['YMD'].astype(str)

df['Y'] = df['YMD'].map(lambda x:x[:4]).astype('int16')
df['M'] = df['YMD'].map(lambda x:x[4:6]).astype('int8')
df['D'] = df['YMD'].map(lambda x:x[6:]).astype('int8')
df['YM'] = df['YMD'].map(lambda x:x[:6]).astype('int32')

In [83]:
# 20년 데이터만 있는 품목 제외

df = df[~df['Product'].isin(['즉석밥', '두부', '김치', '고추장', '된장', '간장', '맛김(조미김)', '콩나물'])]

In [84]:
df.loc[df["Product"]=="피마늘", "Product"] = "마늘"

In [85]:
# Wholesale : 도매
df_w = df[df['WR']=='도매']

In [86]:
# Retail : 소매
df_r = df[df['WR']=='소매']

In [87]:
product_w = df_w['Product'].unique()
product_w = product_w.tolist()

In [88]:
product_r = df_r['Product'].unique()
product_r = product_r.tolist()

In [89]:
df

,YMD,Region,Product,WR,Grade,Price,Y,M,D,YM
0,20190102,서울,단감,도매,상품,33000.0,2019,1,2,201901
1,20190102,서울,단감,도매,중품,28000.0,2019,1,2,201901
2,20190102,서울,바나나,도매,상품,22000.0,2019,1,2,201901
3,20190102,서울,바나나,도매,중품,17000.0,2019,1,2,201901
4,20190102,서울,참다래,도매,상품,35000.0,2019,1,2,201901
...,...,...,...,...,...,...,...,...,...,...
1014738,20141231,강원,참깨,소매,중품,2980.0,2014,12,31,201412
1014739,20141231,충북,참깨,소매,중품,3240.0,2014,12,31,201412
1014740,20141231,전북,참깨,소매,중품,3000.0,2014,12,31,201412
1014741,20141231,경남,참깨,소매,중품,3500.0,2014,12,31,201412


# 물가지수 파일과 겹치는 품목 확인

In [90]:
df_1 = pd.read_csv(f'{data_path}/소비자물가지수_품목별_연도별_월별_filtered.csv')

In [91]:
df_1['품목별'].unique()

array(['총지수', '농축수산물', '\u3000\u3000\u3000(농산물)곡물',
       '\u3000\u3000\u3000\u3000\u3000\u3000쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000현미',
       '\u3000\u3000\u3000\u3000\u3000\u3000찹쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000보리쌀',
       '\u3000\u3000\u3000\u3000\u3000\u3000콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000땅콩',
       '\u3000\u3000\u3000\u3000\u3000\u3000혼식곡',
       '\u3000\u3000\u3000(농산물)채소',
       '\u3000\u3000\u3000\u3000\u3000\u3000배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000상추',
       '\u3000\u3000\u3000\u3000\u3000\u3000시금치',
       '\u3000\u3000\u3000\u3000\u3000\u3000양배추',
       '\u3000\u3000\u3000\u3000\u3000\u3000미나리',
       '\u3000\u3000\u3000\u3000\u3000\u3000깻잎',
       '\u3000\u3000\u3000\u3000\u3000\u3000부추',
       '\u3000\u3000\u3000\u3000\u3000\u3000무',
       '\u3000\u3000\u3000\u3000\u3000\u3000열무',
       '\u3000\u3000\u3000\u3000\u3000\u3000당근',
       '\u3000\u3000\u3000\u3000\u3000\u3000감자',
       '\u3000\u3000\u3000\u

In [92]:
# replace로 \u3000 값이 사라지지 않는다.. 이유는 모르겠음
df_1['품목별'] = df_1['품목별'].replace('\u3000', '')

In [93]:
df_1['품목별'] = df_1['품목별'].map(lambda x:x.strip(('\u3000')))

In [94]:
df['Product'].unique()

array(['단감', '바나나', '참다래', '파인애플', '오렌지', '방울토마토', '고등어', '갈치', '명태',
       '물오징어', '건멸치', '북어', '건오징어', '김', '건미역', '굴', '고구마', '감자', '배추',
       '양배추', '시금치', '상추', '얼갈이배추', '수박', '오이', '호박', '토마토', '딸기', '무',
       '당근', '열무', '건고추', '풋고추', '붉은고추', '마늘', '양파', '파', '생강', '미나리',
       '깻잎', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '사과', '배',
       '레몬', '망고', '전복', '새우', '깐마늘(국산)', '감귤', '포도', '쌀', '찹쌀', '콩', '팥',
       '녹두', '메밀', '참깨', '들깨', '땅콩', '쇠고기', '돼지고기', '닭고기', '계란', '수입조기',
       '호두', '아몬드', '건포도', '건블루베리', '고춧가루', '멸치액젓', '갓', '새우젓', '굵은소금',
       '꽁치', '우유', '부추', '참외', '체리', '복숭아', '가지', '자몽', '깐마늘(수입)', '브로콜리',
       '청경채', '케일', '양송이버섯', '표고버섯', '귀리', '보리', '수수', '율무', '양상추', '혼합곡'],
      dtype=object)

# 농수산물 데이터와 주요경제지표 합치기

In [95]:
file = f"{data_path}/주요경제지표_20052021.xlsx"
df_mei = pd.read_excel(file, engine="openpyxl")
print(df_mei.shape)
display(df_mei.head(df_mei.shape[0]))

(17, 11)


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,연도,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,2021,110.64,108.050,88302.7,-154.3,463118.360,644400.370,615093.450,1144.42,3.7,0.61
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08


In [96]:
df_mei = df_mei[df_mei["연도"] <= 2020]

In [97]:
df_mei = df_mei.rename(columns={"연도" : "Y"})
df_mei

,Y,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
1,2020,104.01,105.420,75275.7,-386.3,443098.120,512788.730,467548.715,1180.05,3.9,0.70
2,2019,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.80,3.8,1.59
3,2018,104.46,104.450,76408.5,316.7,403694.264,604859.657,535202.428,1118.10,3.8,1.52
4,2017,102.53,102.930,78460.0,-26.8,389266.683,573694.421,478478.296,1071.40,3.7,1.26
5,2016,99.11,100.970,98677.4,-46.2,371101.605,495425.934,406192.887,1208.50,3.7,1.34
6,2015,100.95,100.000,105939.6,-60.2,367961.865,526756.498,436498.973,1172.00,3.6,1.65
7,2014,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.20,3.5,2.34
8,2013,105.73,98.048,81148.2,-27.0,346459.569,559632.400,515585.500,1055.30,3.1,2.59
9,2012,107.45,96.789,50835.0,-41.7,326968.393,547869.800,519584.500,1071.10,3.2,3.08
10,2011,106.71,94.717,18655.8,-112.0,306402.488,555213.700,524413.100,1153.30,3.4,3.09


In [220]:
df_market = pd.merge(df, df_mei)
df_market

,YMD,Region,Product,WR,Grade,Price,Y,M,D,YM,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
0,20190102,서울,단감,도매,상품,33000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
1,20190102,서울,단감,도매,중품,28000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
2,20190102,서울,바나나,도매,상품,22000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
3,20190102,서울,바나나,도매,중품,17000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
4,20190102,서울,참다래,도매,상품,35000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14237905,20141231,강원,참깨,소매,중품,2980.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34
14237906,20141231,충북,참깨,소매,중품,3240.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34
14237907,20141231,전북,참깨,소매,중품,3000.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34
14237908,20141231,경남,참깨,소매,중품,3500.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34


In [221]:
crop_list = [
    "호박",
    "고구마",
    "양파",
    "무",
    "배",
    "배추",
    "마늘",
    "양배추",
    "시금치",
    "콩",
    "땅콩",
    "풋고추",
    "오이",
    "깻잎",
    "생강",
    "사과",
    "포도",
    "당근",
    "복숭아",
    "미나리",
    "감자",
    "열무",
    "쌀",
]

len(crop_list)

23

In [222]:
df_market = df_market[df_market["Product"].isin(crop_list)]
df_market

,YMD,Region,Product,WR,Grade,Price,Y,M,D,YM,생산자물가지수(2010=100),소비자물가지수(2015=100),경상수지(백만US$),자본수지(백만US$),외환보유액(백만US$),수출액(백만US$),수입액(백만US$),환율(원/US$),실업률(%),콜금리(연%)
27,20190102,서울,고구마,도매,상품,41000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
28,20190102,서울,고구마,도매,중품,36000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
29,20190102,서울,감자,도매,상품,46000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
30,20190102,서울,감자,도매,중품,36000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
31,20190102,서울,감자,도매,상품,50000.0,2019,1,2,201901,104.48,104.850,59670.1,-169.3,408816.146,542232.610,503342.947,1157.8,3.8,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14237679,20141231,서울,배,도매,중품,30000.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34
14237680,20141231,부산,배,도매,중품,30000.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34
14237681,20141231,대구,배,도매,중품,26000.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34
14237682,20141231,광주,배,도매,중품,25000.0,2014,12,31,201412,105.17,99.298,84374.0,-8.9,363592.748,572664.602,525514.506,1099.2,3.5,2.34


In [101]:
df_market["Product"].nunique()

23

In [102]:
set(crop_list) - set(df_market["Product"])

set()

In [196]:
file = f"{data_path}/생산자물가지수_품목별_연도별_월별.xlsx"
df2 = pd.read_excel(file, engine="openpyxl")
# df2 = df2.rename(columns={'계정코드별':'Product'})
# df2["Product"] = df2["Product"].map(lambda x : x.split()[0])
# df2 = df2.replace("-", np.NaN)
# df2 = df2.dropna(axis=0)
# df2 = df2[df2["Product"].isin(crop_list)]
# for i in df2["Product"]:
#     df2["Product"] = df2["Product"].replace(i,f"생산자물가지수_{i}")

# for i in df2.columns:
#     df2.columns = df2.columns.map(lambda x: x.replace(".",""))
    
# df2.stack()

df2

,계정코드별,2005.01,2005.02,2005.03,2005.04,2005.05,2005.06,2005.07,2005.08,2005.09,...,2022.02,2022.03,2022.04,2022.05,2022.06,2022.07,2022.08,2022.09,2022.10,2022.11
0,쌀 (2015=100),101.54,102.03,102.34,102.24,102.34,102.43,102.43,102.03,98.28,...,127.81,125.83,121.04,117.76,115.19,112.30,107.45,103.12,118.95,118.39
1,보리쌀 (2015=100),58.37,58.37,58.37,58.37,58.37,58.37,58.37,58.37,58.37,...,69.32,69.32,69.32,69.32,72.15,76.28,76.28,76.28,76.28,76.28
2,콩 (2015=100),113.85,116.71,124.75,125.41,123.54,118.36,115.07,110.21,105.15,...,147.85,148.00,147.78,150.50,151.84,151.53,151.53,151.53,152.97,147.21
3,감자 (2015=100),71.15,81.05,84.32,93.61,83.21,43.65,29.87,39.66,37.56,...,137.91,151.95,220.30,197.62,131.95,131.27,142.17,142.83,140.36,150.93
4,고구마 (2015=100),53.12,56.55,58.77,67.39,68.48,68.48,68.48,65.73,51.93,...,106.56,102.67,106.02,103.66,104.20,93.86,99.49,94.70,86.18,91.90
5,배추 (2015=100),32.83,42.02,43.85,40.1,37.52,43.65,65.23,74.08,112.14,...,170.18,194.07,216.60,237.59,191.57,281.64,372.02,657.60,252.06,138.34
6,시금치 (2015=100),48.07,62.09,50.1,45.3,45.16,51.59,61.75,61.75,135.95,...,152.30,118.20,96.31,107.70,132.55,402.91,531.36,337.04,123.26,69.37
7,상추 (2015=100),38.65,37.75,40.94,42.96,34.35,30.12,103.81,124.35,120.81,...,79.72,77.45,90.34,80.79,133.85,330.06,298.40,169.75,94.79,59.05
8,부추 (2015=100),-,-,-,-,-,-,-,-,-,...,168.69,158.35,144.07,85.82,73.01,135.51,169.85,138.23,156.79,179.70
9,수박 (2015=100),46.3,46.3,46.3,46.3,46.3,40.1,34.53,34.81,35.66,...,139.76,143.50,147.74,176.95,142.61,191.15,205.62,140.60,122.22,129.74


In [209]:
df3 = pd.read_csv(f"{data_path}/소비자물가지수_품목별_연도별_월별_filtered.csv")
df3 = df3.replace("-", np.NaN)
# df3 = df3.rename(columns={'품목별':'YM'}).set_index("YM", drop=True)
# df3 = df3.dropna(axis="rows")ㅋㅋㅋㅋㅋ짜증나게하지마세ㅕㅇ 
# # df3 = df3.drop(index=[0,1,2,10,38])
# # df3["YM"] = df3["YM"].map(lambda x: x.strip())
# # df3 = df3[df3["YM"].isin(crop_list)]
# # for i in df3["YM"]:
# #     df3["YM"] = df3["YM"].replace(i,f"소비자물가지수_{i}")

# for i in df3.columns:
#     df3.columns = df3.columns.map(lambda x: x.replace(".",""))
        
# pd.DataFrame(df3.stack()) 
df3

,품목별,2005.01,2005.02,2005.03,2005.04,2005.05,2005.06,2005.07,2005.08,2005.09,...,2022.02,2022.03,2022.04,2022.05,2022.06,2022.07,2022.08,2022.09,2022.10,2022.11
0,총지수,73.594,73.892,74.338,74.413,74.338,74.114,74.413,74.636,75.008,...,105.30,106.06,106.85,107.56,108.22,108.74,108.62,108.93,109.21,109.10
1,농축수산물,62.157,64.050,64.302,64.681,63.482,61.526,62.788,63.293,65.249,...,111.72,109.89,110.42,112.45,111.80,114.40,116.74,118.99,114.68,109.78
2,(농산물)곡물,76.091,76.015,75.866,75.641,76.015,76.015,75.866,75.716,75.042,...,103.59,102.59,101.13,99.72,98.51,97.32,95.47,93.93,94.40,97.81
3,쌀,75.464,75.539,75.391,75.093,75.242,75.316,75.168,75.019,74.277,...,102.87,101.90,99.94,98.23,96.77,95.23,92.80,90.34,91.08,96.16
4,현미,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.62,105.34,105.25,104.03,102.93,102.33,101.34,101.72,101.01,100.42
5,찹쌀,78.245,76.985,76.120,77.694,81.236,81.394,80.212,80.921,80.764,...,103.63,101.56,100.81,99.76,98.74,97.55,95.74,97.04,96.87,95.84
6,보리쌀,61.778,61.839,61.778,61.839,61.901,61.901,61.839,61.716,61.654,...,101.50,102.44,103.10,102.99,103.55,106.59,108.89,109.55,110.14,111.09
7,콩,68.828,68.560,68.022,68.090,67.755,67.217,67.486,67.149,66.880,...,107.30,106.71,105.04,104.34,103.63,103.04,103.01,102.52,102.75,102.03
8,땅콩,53.740,53.166,52.225,52.172,53.061,52.225,53.532,52.800,52.905,...,105.40,105.95,106.17,107.06,106.73,107.11,106.28,106.40,106.03,106.44
9,혼식곡,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,104.67,103.52,104.74,104.34,104.35,103.82,103.72,104.20,104.08,105.56


In [181]:
# 이게 우신님꺼
df_eco = pd.concat([df2, df3], axis = 0).set_index("YM").T
df_eco

YM,생산자물가지수_쌀,생산자물가지수_콩,생산자물가지수_감자,생산자물가지수_고구마,생산자물가지수_배추,생산자물가지수_시금치,생산자물가지수_오이,생산자물가지수_호박,생산자물가지수_무,생산자물가지수_풋고추,...,소비자물가지수_오이,소비자물가지수_풋고추,소비자물가지수_호박,소비자물가지수_양파,소비자물가지수_마늘,소비자물가지수_생강,소비자물가지수_사과,소비자물가지수_배,소비자물가지수_복숭아,소비자물가지수_포도
200501,101.54,113.85,71.15,53.12,32.83,48.07,115.9,86.88,35.49,76.99,...,51.792,57.365,79.112,55.449,58.270,47.425,79.064,68.187,89.542,107.898
200502,102.03,116.71,81.05,56.55,42.02,62.09,137.26,93.54,35.49,115.65,...,64.107,73.732,95.253,62.842,58.270,49.972,93.523,73.784,89.542,107.898
200503,102.34,124.75,84.32,58.77,43.85,50.1,108.7,88.17,50.79,122.01,...,53.364,81.833,79.182,72.111,59.076,51.460,96.489,76.547,89.542,107.898
200504,102.24,125.41,93.61,67.39,40.1,45.3,67.88,57.13,55.29,85.71,...,43.430,62.205,60.547,78.677,59.479,52.325,100.474,77.564,89.542,107.898
200505,102.34,123.54,83.21,68.48,37.52,45.16,51.66,34.76,56.08,59.21,...,38.819,50.514,42.743,59.918,58.730,52.277,101.401,77.564,89.542,107.898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202207,112.3,151.53,131.27,93.86,281.64,402.91,174.41,147.87,227.61,204.03,...,148.97,122.52,119.17,97.9,146.5,63.67,112.07,128.98,137.17,154.93
202208,107.45,151.53,142.17,99.49,372.02,531.36,181.06,210.9,276.95,147.23,...,157.4,118.3,139.6,101.09,146.55,64.74,117.77,124.17,127.86,158.97
202209,103.12,151.53,142.83,94.7,657.6,337.04,147.16,152.01,369.7,196.88,...,139.27,133.75,150.09,105.93,144.96,68.26,105.89,118.03,136.4,138.43
202210,118.95,152.97,140.36,86.18,252.06,123.26,108.19,108.84,316.0,147.28,...,124.6,104.78,101.54,109.17,144.46,73.11,95.2,104.94,130.99,131.76


In [157]:
df_eco["YM"]

KeyError: 'YM'

In [156]:
pd.merge(df_market, df_eco, on="YM")

KeyError: 'YM'